In [1]:
from scipy.io import loadmat

filename = 'ex4/ex4data1.mat'

data = loadmat(filename)
X = data['X']
y = data['y']

In [2]:
import numpy as np
import matplotlib.pyplot as plt

X_shape = X.shape
n = int(np.sqrt(X_shape[1]))
X_reshaped = X.reshape(X_shape[0],n,n)
X_flipped = np.swapaxes(X_reshaped, 1, 2)

width_to_plot = 10
number_to_plot = np.square(width_to_plot)

selected_indices = np.random.choice(X_shape[0], number_to_plot, replace = False)
selected_cases = X_flipped[selected_indices, :, :]

Y = np.zeros((X_shape[0], 10))
for i in range(X_shape[0]):
    Y[i, int(y[i]%10)] = 1



In [3]:
import torch
import torch.utils.data as torch_data
from torch.utils.data import DataLoader

X_tensor = torch.tensor(X).to(torch.float)
Y_tensor = torch.tensor(Y).to(torch.float)

r_train = 0.5
train_size = int(r_train * len(X_tensor))
test_size = len(X_tensor) - train_size

train_dataset, test_dataset = torch_data.random_split(
    dataset = torch_data.TensorDataset(X_tensor, Y_tensor), 
    lengths = [train_size, test_size]
)

batch_size = 2500
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size)

c:\Users\yufengliu812\anaconda3\envs\transformer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(400, 200)
        self.fc2 = nn.Linear(200,100)
        self.fc3 = nn.Linear(100,10)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim = 1)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f"Using device: {device}")

net = Net()
net.to(device)

Using device: cuda


Net(
  (fc1): Linear(in_features=400, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

In [5]:
import time 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01, momentum= 0.9)

num_epochs = 10000

for epoch in range(num_epochs):
    net.train()
    running_loss = 0.0
    for inputs, labels in train_loader:

        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if (epoch + 1)%500 ==0 or epoch ==0:
        avg_training_loss = running_loss/len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_training_loss:.4f}")


        net.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs, 1)

                labels = torch.argmax(labels, dim = 1)

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f"Validation Accuracy: {val_accuracy:.2f}%")







Epoch [1/10000], Loss: 2.3502
Validation Accuracy: 9.68%
Epoch [500/10000], Loss: 2.2611
Validation Accuracy: 21.48%
Epoch [1000/10000], Loss: 1.5583
Validation Accuracy: 57.20%
Epoch [1500/10000], Loss: 0.8427
Validation Accuracy: 73.64%
Epoch [2000/10000], Loss: 0.6368
Validation Accuracy: 80.96%
Epoch [2500/10000], Loss: 0.4873
Validation Accuracy: 84.24%
Epoch [3000/10000], Loss: 0.3996
Validation Accuracy: 86.56%
Epoch [3500/10000], Loss: 0.3478
Validation Accuracy: 88.16%
Epoch [4000/10000], Loss: 0.3121
Validation Accuracy: 88.84%
Epoch [4500/10000], Loss: 0.2846
Validation Accuracy: 88.80%
Epoch [5000/10000], Loss: 0.2620
Validation Accuracy: 89.48%
Epoch [5500/10000], Loss: 0.2425
Validation Accuracy: 89.68%
Epoch [6000/10000], Loss: 0.2251
Validation Accuracy: 90.00%
Epoch [6500/10000], Loss: 0.2093
Validation Accuracy: 90.04%
Epoch [7000/10000], Loss: 0.1948
Validation Accuracy: 90.24%
Epoch [7500/10000], Loss: 0.1814
Validation Accuracy: 90.44%
Epoch [8000/10000], Loss: 0.1